# Agentic AI Workshop — Part 1: Basic LLM Calls, Streaming, and Chat

This notebook demonstrates:
- Text-in → Text-out with the OpenAI Python SDK
- Text → Image generation
- Text → Audio (TTS)
- A minimal Gradio UI for an LLM call
- Streaming responses
- A simple multi‑turn chat

> **Prereqs**
> - `pip install openai gradio`  
> - Set your API key in the environment as `OPENAI_API_KEY` (recommended).


In [ ]:
import os
from PIL import Image
from openai import OpenAI
import gradio as gr
import sys
import os
import requests
from IPython.display import Image, display, Audio
from pathlib import Path

sys.path.append('../utils')
from helpers import load_env

In [ ]:
# Load and verify API keys
load_env(api_key_type="OPENAI_API_KEY")

In [ ]:
# Outputs directory
OUTPUTS_DIR = './../outputs/'
os.makedirs(OUTPUTS_DIR, exist_ok=True)

## # Initialize OpenAI client

In [ ]:
# Initialize OpenAI client
# This will automatically use the OPENAI_API_KEY environment variable that was loaded
# You can also explicitly pass the key with: OpenAI(api_key="sk-...")
client = OpenAI()  # Uses OPENAI_API_KEY from environment
print("Client ready.")

## 1. Test basic text completion

In [ ]:
def get_single_turn_completion(prompt: str) -> str:
    """Get a single-turn completion from the LLM (no conversation history)"""
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    
    return response.choices[0].message.content

prompt = "Write a friendly, two-sentence definition of Retrieval-Augmented Generation."

print("\nUser Prompt:")
print("-" * 40)
print(prompt)
print("-" * 40)

result = get_single_turn_completion(prompt)

print("\n LLM Response:")
print("-" * 40)
print(result) 
print("-" * 40)


## 2. Streaming Response

In [ ]:
def stream_response(prompt):
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        stream=True
    )
    
    response_text = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            chunk_text = chunk.choices[0].delta.content
            response_text += chunk_text
            print(chunk_text, end="", flush=True)
    return response_text

# Test streaming
prompt = "Write a short 300 words story about a robot who discovers emotions."
print("\nUser Prompt:")
print("-" * 40)
print(prompt)
print("-" * 40)

print("\nStreaming response:")
print("-" * 40)
response = stream_response(prompt)
print("\n" + "-" * 40)


## 3. Image Generation - Text to Image


In [ ]:
def generate_image(prompt):
    """Generate an image from text"""
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size="1024x1024",
        quality="standard", 
        n=1,
    )
    return response.data[0].url

# Test image generation
image_url = generate_image(
    "Flat image of a helpful robot librarian fetching a book titled 'RAG' "
    "from a bookshelf. Teal shade.")

# Download the image
image_response = requests.get(image_url, timeout=20)

# Print image size
image_size = len(image_response.content)
print(f"Image size: {image_size:,} bytes")

# Save the image
output_path = os.path.join(OUTPUTS_DIR, 'generated_image.png')
with open(output_path, 'wb') as f:
    f.write(image_response.content)
print(f"Image saved to: {output_path}")

# Display the image
display(Image(url=image_url))

## 4. Audio Generation - Text to Speech


In [ ]:
def generate_speech(text, voice="fable"):
    """
    Generate speech from text
    
    Args:
        text (str): The text to generate speech from
        voice (str): The voice to use for the speech (default is "alloy")
                     Options: "alloy", "echo", "fable", "onyx", "nova", "shimmer"

    Returns:
        bytes: The generated audio content
    """
    response = client.audio.speech.create(
        model="tts-1",
        voice=voice,
        input=text
    )
    return response.content

# Test speech generation
audio_content = generate_speech(
    "Hello! Welcome to the Agentic AI Workshop. "
    "Let's build some amazing AI applications together!")

# Print audio size
print(f"Generated audio content: {len(audio_content)} bytes")

# Save the audio
audio_fpath = Path(OUTPUTS_DIR) / 'generated_audio.mp3'
with open(audio_fpath, 'wb') as f:
    f.write(audio_content)
print(f"Audio saved to: {audio_fpath}")

# Play inline
Audio(audio_content, autoplay=True)

## 5. Gradio UI for Interactive Testing


### OpenAI Message Format:
- List of message objects with "role" and "content" fields
- Roles: "system" (instructions), "user" (human input), "assistant" (AI response)
- Example: [{"role": "user", "content": "Hello"}, {"role": "assistant", "content": "Hi!"}]

Roles can be system, user, assistant, tool

### Gradio Message Format:
- For basic Interface: Single input string passed directly to function
- For ChatInterface: Tuple of (current_message, list_of_message_pairs)
- Message pairs are (user_message, assistant_message) tuples

In [ ]:
# Create a simple Gradio interface for text generation
def chat_with_llm(message):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

# Create Gradio interface
iface = gr.Interface(
    fn=chat_with_llm,
    inputs="text",
    outputs="text",
    title="Basic LLM Chat",
    description="A simple chat interface with GPT-3.5-turbo",
    examples=[
        "Tell me a joke",
        "Explain quantum computing in simple terms",
        "Write a haiku about AI"
    ]
)

# Launch the interface
iface.launch()


## 6. Multi-turn Chat Conversation


In [59]:

# Multi-turn chat with conversation history
def chat_with_history(message, history):
    """Chat function with conversation history."""
    # Start with system message if this is the first turn
    if not history:
        messages = [{"role": "system", "content": "You are a helpful AI assistant."}]
    else:
        messages = []
    
    # Add conversation history
    messages.extend(history)
    
    # Add current user message
    messages.append({"role": "user", "content": message})
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )
    
    return response.choices[0].message.content

# Create advanced chat interface
chat_interface = gr.ChatInterface(
    fn=chat_with_history,
    title="Advanced AI Chat", 
    description="A multi-turn chat interface with conversation memory - maintains full chat history",
    examples=[
        "Hello! What can you help me with?",
        "Tell me about machine learning", 
        "Can you write a Python function?"
    ],
    cache_examples=False,
    type="messages"  # Use the new messages format instead of deprecated tuples
)

# Launch the advanced interface
chat_interface.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


# Exercises 

## EXERCISE 1: Add Streaming to Multi-Turn Chatbot

Your task: Modify the chat function to stream responses token-by-token instead of 
waiting for the complete response.

## EXERCISE 2: Add Text-to-Speech to Your Chatbot

Your task: Create a chatbot that streams the text response AND generates audio 
of the AI's response. The AI Assistant should speak its response while also streaming it. 